In [1]:
import pandas as pd
import my_predictor as mp
import gdi_loader as gdi
from sklearn import svm
import bert_swiss_lm as bsl
import itertools
import torch
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [2]:
my_predictor = mp.MyPredictor("train_embedding_bert_swiss_lm.feather", "dev_embedding_bert_swiss_lm.feather", None)

In [4]:
my_predictor.fit()

In [5]:
my_predictor.predict_all()

Training set:
              precision    recall  f1-score   support

          BE       0.79      0.81      0.80      3750
          BS       0.74      0.72      0.73      3269
          LU       0.77      0.64      0.70      3390
          ZH       0.74      0.86      0.80      3870

    accuracy                           0.76     14279
   macro avg       0.76      0.76      0.76     14279
weighted avg       0.76      0.76      0.76     14279

Dev set:
              precision    recall  f1-score   support

          BE       0.63      0.62      0.63      1053
          BS       0.71      0.58      0.64      1528
          LU       0.64      0.49      0.55      1017
          ZH       0.56      0.87      0.68       932

    accuracy                           0.63      4530
   macro avg       0.63      0.64      0.63      4530
weighted avg       0.64      0.63      0.63      4530



In [3]:
my_predictor_pooler = mp.MyPredictor("train_embedding_bert_swiss_lm.feather",
                                     "dev_embedding_bert_swiss_lm.feather",
                                     None,
                                     ["sentence_embedding_first_state", "audio"],
                                     audio_length=10)

my_predictor_pooler.fit()
my_predictor_pooler.predict_all()

Training set:
              precision    recall  f1-score   support

          BE       0.78      0.83      0.81      3750
          BS       0.74      0.73      0.73      3269
          LU       0.74      0.67      0.71      3390
          ZH       0.80      0.82      0.81      3870

    accuracy                           0.77     14279
   macro avg       0.76      0.76      0.76     14279
weighted avg       0.77      0.77      0.77     14279

Dev set:
              precision    recall  f1-score   support

          BE       0.57      0.60      0.58      1053
          BS       0.68      0.54      0.60      1528
          LU       0.53      0.45      0.49      1017
          ZH       0.57      0.81      0.67       932

    accuracy                           0.59      4530
   macro avg       0.59      0.60      0.58      4530
weighted avg       0.60      0.59      0.59      4530



In [8]:
my_predictor_with_audio = mp.MyPredictor("train_embedding_bert_swiss_lm.feather", "dev_embedding_bert_swiss_lm.feather", None, "sentence_embedding_pooler")
my_predictor_with_audio.fit()
my_predictor_with_audio.predict_all()

KeyError: 's'

In [3]:
my_predictor_pooler.fit()

In [4]:
my_predictor_pooler.predict_all()

Training set:
              precision    recall  f1-score   support

          BE       0.74      0.75      0.74      3750
          BS       0.66      0.63      0.64      3269
          LU       0.68      0.55      0.61      3390
          ZH       0.68      0.81      0.74      3870

    accuracy                           0.69     14279
   macro avg       0.69      0.68      0.68     14279
weighted avg       0.69      0.69      0.69     14279

Dev set:
              precision    recall  f1-score   support

          BE       0.61      0.61      0.61      1053
          BS       0.71      0.57      0.64      1528
          LU       0.66      0.49      0.56      1017
          ZH       0.54      0.87      0.67       932

    accuracy                           0.62      4530
   macro avg       0.63      0.64      0.62      4530
weighted avg       0.64      0.62      0.62      4530



In [8]:
prediction_dev = clf.predict(embdedings_numpy_dev)
print(prediction_dev)
print(lables_dev)

['LU' 'BS' 'LU' 'LU' 'BE' 'ZH' 'ZH' 'LU' 'BS' 'LU' 'LU' 'BE' 'ZH' 'LU'
 'LU' 'BE' 'LU' 'BE' 'LU' 'LU' 'BS' 'BE' 'BS' 'ZH' 'BS' 'BE' 'ZH' 'BS'
 'BE' 'BE' 'BS' 'BS' 'BS' 'LU' 'ZH' 'BS' 'ZH' 'ZH' 'LU' 'LU' 'LU' 'ZH'
 'ZH' 'BS' 'BE' 'ZH' 'LU' 'BS' 'LU' 'ZH' 'BE' 'ZH' 'BS' 'ZH' 'BS' 'ZH'
 'ZH' 'BE' 'BE' 'BS' 'LU' 'BS' 'BE' 'ZH' 'ZH' 'ZH' 'LU' 'BS' 'BS' 'LU'
 'BE' 'BS' 'ZH' 'ZH' 'ZH' 'ZH' 'BE' 'BE' 'BS' 'ZH' 'ZH' 'BE' 'BS' 'ZH'
 'BS' 'BS' 'BS' 'BS' 'BE' 'BS' 'ZH' 'BS' 'ZH' 'ZH' 'LU' 'ZH' 'BS' 'ZH'
 'BS' 'ZH' 'BS' 'BS' 'LU' 'BS' 'BE' 'BS' 'LU' 'ZH' 'ZH' 'BS' 'BE' 'LU'
 'ZH' 'ZH' 'ZH' 'BE' 'BE' 'ZH' 'BS' 'ZH' 'BE' 'LU' 'BS' 'BE' 'BS' 'BS'
 'BE' 'BS' 'BS' 'BS' 'LU' 'BE' 'ZH' 'LU' 'LU' 'BE' 'BE' 'LU' 'BE' 'ZH'
 'BS' 'BE' 'BS' 'BS' 'BE' 'ZH' 'ZH' 'ZH' 'ZH' 'BE' 'BS' 'BS' 'ZH' 'ZH'
 'LU' 'ZH' 'ZH' 'BS' 'BS' 'BS' 'BS' 'LU' 'BE' 'LU' 'BE' 'BS' 'BE' 'BE'
 'ZH' 'ZH' 'LU' 'ZH' 'ZH' 'ZH' 'ZH' 'BE' 'BS' 'BE' 'BS' 'LU' 'ZH' 'LU'
 'BE' 'BS' 'ZH' 'LU' 'BS' 'BE' 'BE' 'ZH' 'BE' 'BS' 'BE' 'BS' 'BS' 'ZH'
 'BE' 

In [13]:
print(precision_score(lables_dev, prediction_dev, average="macro"))
print(classification_report(lables_dev, prediction_dev))

0.6147505478251539
              precision    recall  f1-score   support

          BE       0.60      0.62      0.61       242
          BS       0.69      0.52      0.60       335
          LU       0.69      0.49      0.57       235
          ZH       0.48      0.84      0.61       188

    accuracy                           0.60      1000
   macro avg       0.61      0.62      0.60      1000
weighted avg       0.63      0.60      0.60      1000

